<a href="https://colab.research.google.com/github/fagc/ScrappingData/blob/master/Colab_DR_Jobs_Scraping_September_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import datetime
import re

#####LinkedIn

rawdata = pd.read_csv(r"C:\Users\Gav\Anaconda3\Projects\Jobs_scraping\linkedin-jobs-2021-08_02-predicted.csv")
data = rawdata[rawdata['description'].notna()]

###LinkedIn cleaning and keyword fitlering, date filtering

unique = data.duplicated(subset=['description'])
data['duplicated'] = unique

##job description scraping
targets = ['secur', 'risk manag', 'cyber', 'Risk Manag', 'SOC', 'Secur', 'Cyber', 'Penetration', 'incident response', 'Threat Intelligence', 'Vulnerability', 'Detection', 'Incident Response', 'Threat']
keyword = data.description.apply(lambda sentence: any(word in sentence for word in targets))
data['keyword_scraping'] = keyword

##job title scraping
title_targets = ['Retire', 'People', 'Benefit', 'Motor', 'Sourc', 'Area', 'Marine', 'hand', 'Rating', 'man', 'Trade', 'Propos', 'Learn', 'Medi', 'Operator', 'Pric', 'Operating', 'Librar', 'Regis', 'Know', 'Case', 'Appeal', 'Territory', 'House', 'Construct' ,'Elect', 'Fire ', 'Robot', 'Kni', 'Bagel', 'Custod', 'Sneak', 'Image', 'Service', 'Value', 'Real', 'Sell', 'Veg', 'Energy', 'Wine', 'ASSOC', 'Bake', 'Drive', 'Rural', 'Tast', 'Dish', 'Janit', 'Carpe', 'Pain', 'Investor', 'Port', 'Target', 'Barista', 'Chef', 'Cashier', 'Hospital', 'Cook', 'Stock', 'Food', 'Mile', 'Silicon', 'Neuro', 'Fund', 'Home', 'Credit', 'HR', 'Banking', 'Lube', 'Payroll', 'Cabin', 'SALES', 'Wealth', 'CLERK', 'Care', 'Warehouse', 'Valet', 'Parking', 'Merch', 'Property', 'Resident', 'Banker', 'Store', 'Supplier', 'Assets', 'Data Sci', 'Land', 'Restaurant', 'Pharma', 'Clerk', 'Attend', 'Pediatric', 'Valet', 'Store', 'Concierge', 'Workplace', 'Sale', 'Asset', 'Alcohol', 'Front', 'Loss Prevention', 'Guest', 'Buyer', 'Distrib', 'Clinic', 'Key', 'Heart', 'Material', 'Ling', 'Oral', 'Quality', 'Maint', 'Strategy', 'Field', 'Water', 'Pack', 'bound', 'Product', 'Electr', 'Counsel', 'Chem', 'Lab', 'Therapy', 'Logistics', 'Reward', 'Nurse', 'Paralegal' 'Hous', 'Media', 'Talent', 'Accounting', 'Geo', 'Loan', 'Equity', 'Asset Protection', 'Demand', 'Art', 'Video', 'Consumer', 'Dairy', 'Financ', 'Manufacturing', 'Safety', 'Health', 'Design', 'Repair', 'Software Dev', 'Software Engineer', 'Full Stack', 'Market' ,'Plant', 'Sales', 'Actuar', 'Claim', 'Legal', 'Human Resources', 'Client', 'Customer']
title_keyword = data.job_title.apply(lambda sentence: any(word in sentence for word in title_targets))
data['title_keyword_scraping'] = title_keyword


#date filtering
start = datetime.date(2021, 6, 1)

def two_months_date(df):
        
    if (df['posted_date'] > start):
        return 'True'
    elif (df['posted_date'] < start):
        return 'False'
    else:
        return 'True'
    
data[['posted_date']].replace('(/)','-',regex=True, inplace = True)
data['posted_date'] = pd.to_datetime(data['posted_date']).dt.date
data['date_criteria'] = data.apply(two_months_date, axis = 1)
data['posted_date'] = data[['posted_date']].astype(str)

#####INDEED

rawdata_indeed = pd.read_csv(r"C:\Users\Gav\Anaconda3\Projects\Jobs_scraping\indeed-jobs-2021-08-predicted.csv")
data_indeed = rawdata_indeed[rawdata_indeed['description'].notna()]
data_indeed['seniority_level'] = 'Not Applicable'

###INDEED cleaning and keyword fitlering

unique = data_indeed.duplicated(subset=['description'])
data_indeed['duplicated'] = unique

##job description scraping
keyword = data_indeed.description.apply(lambda sentence: any(word in sentence for word in targets))
data_indeed['keyword_scraping'] = keyword

##job title scraping
title_keyword_indeed = data_indeed.job_title.apply(lambda sentence: any(word in sentence for word in title_targets))
data_indeed['title_keyword_scraping'] = title_keyword_indeed

#date filtering

data_indeed[['posted_date']].replace('(/)','-',regex=True, inplace = True)
data_indeed['posted_date'] = pd.to_datetime(data_indeed['posted_date']).dt.date
data_indeed['date_criteria'] = data_indeed.apply(two_months_date, axis = 1)
data_indeed['posted_date'] = data_indeed[['posted_date']].astype(str)


##COMBINED

data_all = data.append(data_indeed, sort=False)

###reclass Cyber=1
check_cyber = data_all.job_title.apply(lambda sentence: any(word in sentence for word in targets))
data_all['without_doubt_cyber'] = check_cyber

def check_cyber_f(df):

    if (df['Cyber=1'] == 0 and df['without_doubt_cyber'] == True):
        return 1
    else:
        return df['Cyber=1']
    
data_all['Cyber=1'] = data_all.apply(check_cyber_f, axis = 1)

###FILTERED CSV

data_filtered = data_all[(data_all['duplicated'] == False) & (data_all['Cyber=1'] == 1) & (data_all['keyword_scraping'] == True) & (data_all['title_keyword_scraping'] == False) & (data_all['date_criteria'] == 'True')]

#'threat intelligence'  'incident response' 
#

###TURN SENIORITY LEVEL INTO JR/MID/SR BREAKDOWN

def seniority_df(df):

    if (df['seniority_level'] == 'Internship' or df['seniority_level'] == 'Entry level'):
        return 'Jr'
    elif (df['seniority_level'] == 'Mid-Senior level' or df['seniority_level'] == 'Associate'):
        return 'Mid'
    elif (df['seniority_level'] == 'Director' or df['seniority_level'] == 'Executive'):
        return 'Sr'
    elif (df['seniority_level'] == 'Not Applicable' or df['seniority_level'] == ''):
        return 'Not Applicable'

data_filtered_2 = data_filtered
data_filtered['seniority'] = data_filtered.apply(seniority_df, axis = 1)

#not applicable classification with job description years of experience#####
import re
import ast

data_filtered['yrs'] = data_filtered['description'].str.findall(r"([\d. +-/]+|[\(\d\). +-/]+|[\w\+\b]+)\s*years of")
raw = data_filtered['yrs']


data_filtered = data_filtered[~data_filtered.index.duplicated(keep='first')]

data_filtered['yrs'] = data_filtered['yrs'].astype(str)\
         .str.replace(".", ' ')\
.str.replace("(", '')\
.str.replace(")", '')\
.str.replace("+", ' ')\
.str.replace("œ", ' ')\
.str.replace("(\d-)", ' ',regex=True)\
.str.replace("(\s-\s\d+)", '',regex=True)\
.str.replace("(-)", '',regex=True)\
.str.replace("(,1\d|,10|ten|Ten)", '10',regex=True)\
.str.replace("(,5|five|œ5|Five)", '5',regex=True)\
.str.replace("eighteen", '18',regex=True)\
.str.replace("(fifteen|,15)", '15',regex=True)\
.str.replace("(Twelve|twelve)", '12',regex=True)\
.str.replace("(,9|nine)", '9',regex=True)\
.str.replace("(,3|three|Three)", '3',regex=True)\
.str.replace("(,8|/8|eight|Eight|œ8)", '8',regex=True)\
.str.replace("(,4|/4|four|Four)", '4',regex=True)\
.str.replace("(,2|/2|two|Two|œ2)", '2',regex=True)\
.str.replace("(,7|/7|seven)", '7',regex=True)\
.str.replace("(,6|six)", '6',regex=True)\
.str.replace(",1", '1',regex=True)\
.str.replace("[a-zA-Z+]", '',regex=True)\
.str.replace("(',', | ','|/|,0|Â)", '')\
.str.replace(" ", '').apply(ast.literal_eval)

data_filtered['yrs'] = data_filtered['yrs'].astype(str)\
         .str.replace(".", ' ')\
.str.replace("+", ' ')\
.str.replace("(\d-)", '',regex=True)\
.str.replace("(-)", '',regex=True)\
.str.replace("(,1\d|,10)", '10',regex=True)\
.str.replace(",5", '5',regex=True)\
.str.replace(",9", '9',regex=True)\
.str.replace(",3", '3',regex=True)\
.str.replace("(,8|/8)", '8',regex=True)\
.str.replace("(,4|/4)", '4',regex=True)\
.str.replace("(,2|/2)", '2',regex=True)\
.str.replace("(,7|/7)", '7',regex=True)\
.str.replace(",6", '6',regex=True)\
.str.replace(",1", '1',regex=True)\
.str.replace("(',', | ','|/|,0)", '')\
.str.replace(",,", ',',regex=True)\
.str.replace(" ", '').apply(ast.literal_eval)

yrs_exp = data_filtered['yrs']

data_filtered['yrs_exp'] = yrs_exp

data_filtered['yrs_exp2'] = np.where(data_filtered['yrs_exp'].astype(str)\
         .str.contains("\d"), data_filtered['yrs_exp'], '0')
data_filtered['yrs_exp2']

seniority_data = data_filtered[(data_filtered['yrs_exp2'] != '0')]
avgtest = seniority_data['yrs_exp2']
avgtest2 = [' '.join(i).split() for i in avgtest]
converted = [[int(num) for num in sub] for sub in avgtest2]
means = [np.mean([el for el in sublist if el > 0] or 0) for sublist in converted]
seniority_data['means'] = means
seniority_data['means'] = seniority_data['means'].astype('int64')

def not_applicable_seniority(df):
        
    if (df['means'] <= 3):
        return 'Jr'
    elif (df['means'] > 3 and df['means'] <= 10):
        return 'Mid'
    elif (df['means'] > 10 and df['means'] <= 25):
        return 'Sr'
    else:
        return 'Not Applicable'
    
seniority_data['means_class'] = seniority_data.apply(not_applicable_seniority, axis = 1)

def not_applicable_seniority_reclass(df):
        
    if (df['seniority'] == 'Not Applicable'):
        return df['means_class']
    else:
        return df['seniority']
    
seniority_data['seniority_v2'] = seniority_data.apply(not_applicable_seniority_reclass, axis = 1)

merged_table = data_filtered.merge(seniority_data[['seniority_v2']], how='left', left_index=True, right_index=True)
merged_table['seniority_v2']= merged_table[['seniority_v2']].fillna(value=0)
#merged_table

def not_applicable_seniority_merge(df):
        
    if (df['seniority_v2'] == 0):
        return df['seniority']
    else:
        return df['seniority_v2']

merged_table['seniority_v3'] = merged_table.apply(not_applicable_seniority_merge, axis = 1)
data_filtered['seniority_v3'] = merged_table['seniority_v3']

#end of seniority classification#####

jr_keywords = ['Intern', 'junior', 'Junior', 'jr', 'Jr', 'Entry']
mid_keywords = ['Head', 'Pro', 'Specialist', 'Expert', 'Manager', 'Consultant', 'Lead', 'Partner', 'Senior', 'Sr', 'II', 'Advisor', 'Principal', 'Associate', 'Supervisor']
sr_keywords = ['Director', 'Officer', 'Chief', 'Executive', 'BISO', 'CISO']

title_keyword_jr = data_filtered.job_title.apply(lambda sentence: any(word in sentence for word in jr_keywords))
data_filtered['jr_title_v4'] = title_keyword_jr

title_keyword_mid = data_filtered.job_title.apply(lambda sentence: any(word in sentence for word in mid_keywords))
data_filtered['mid_title_v4'] = title_keyword_mid

title_keyword_sr = data_filtered.job_title.apply(lambda sentence: any(word in sentence for word in sr_keywords))
data_filtered['sr_title_v4'] = title_keyword_sr

def not_applicable_seniority_title(df):
        
    if (df['seniority_v3'] == 'Not Applicable' and df['jr_title_v4'] == True):
        return 'Jr'
    if (df['seniority_v3'] == 'Not Applicable' and df['mid_title_v4'] == True):
        return 'Mid'
    if (df['seniority_v3'] == 'Not Applicable' and df['sr_title_v4'] == True):
        return 'Sr'
    else:
        return df['seniority_v3']

data_filtered['seniority_v4'] = data_filtered.apply(not_applicable_seniority_title, axis = 1)


######Build table with each CID having one row
summary_table = data_filtered['cid'].drop_duplicates()
summary_table_df = pd.DataFrame(summary_table)

######calculate number  of jobs per cid

df2=data_filtered.set_index('cid')
g = df2.groupby(['cid']).sum()

data_filtered_3 = pd.merge(summary_table_df, g, on="cid", how="inner")
data_filtered_3.drop(columns =['applicant_count', 'duplicate', 'duplicated'])


#####ASSIGN seniority VALUES TO CIDs

jr = data_filtered.groupby(['cid', 'seniority_v4'])[['seniority_v4']].count()

jr3 = jr.unstack(level='seniority_v4', fill_value=0)
jr4 = jr3['seniority_v4']
data_filtered_4 = pd.merge(data_filtered_3, jr4, on="cid", how="left")



###remove unneeded columns
data_filtered_5 = data_filtered_4.drop(columns =['applicant_count', 'Cyber=1', 'jr_title_v4', 'mid_title_v4', 'sr_title_v4', 'duplicate', 'duplicated', 'title_keyword_scraping'])
data_final = data_filtered_5.rename(columns={"keyword_scraping": "total_open_roles"})

### not applicable buckets
data_final['Jr'] = data_final['Jr'] + .35 * data_final['Not Applicable']
data_final['Mid'] = data_final['Mid'] + .60 * data_final['Not Applicable']
data_final['Sr'] = data_final['Sr'] + .05 * data_final['Not Applicable']
data_final['Not Applicable'] = 0

data_finalv2 = data_final.round(decimals=0)
data_finalv2['total_open_roles'] = data_finalv2['Jr'] + data_finalv2['Mid'] + data_finalv2['Sr']

######CALCULATE HC OPEX NUMBERS

data_finalv2['total HC OpEx'] = ((65000 * data_finalv2['Jr'] + 145000 * data_finalv2['Mid'] + 287500 * data_finalv2['Sr'])/12) * 1.35


data_finalv3 = data_finalv2.fillna(0)
data_finalv3